<img src="./images/logo.png" alt="Drawing" style="width: 500px;"/>

<div class="alert alert-block alert-danger">
<b>Important:</b> This exercise requires the completion of <a href="./01.prepare_the_data.ipynb" <b>Exercise 1:</b> Prepare the Data</a></div>

# **Exercise 2:** Connect and Query Data Sources with EzPresto

This exercise will be introduce **EzPresto**, HPE's supercharged Presto distribution on **HPE AI Essentials**. EzPresto is an SQL query engine based on the open-source query engine PrestoDB that is optimized to run federated queries across various data sources. Enterprise applications and data processing engines (such as Spark!) can leverage EzPresto for rapid query performance and prompt insights through federated data access. 

EzPresto is the underlying driver behind all of what you will learn in this exercise, in which you will:

- Connect Data Sources to HPE Private Cloud AI.
- Run SQL queries using the Query Editor.
- Learn about HPE's custom Jupyter Magic commands.
- Run SQL queries on connected Data Sources directly from within a notebook cell.

By the end of this exercise, you will be proficient in directly or indirectly leveraging EzPresto to streamline your data analytics workflows. 

Let's dive in!

## **1. Connecting a Data Source in AI Essentials**

**HPE Private Cloud AI** allows users to connect multiple types of internal and external data sources - from SQL servers to Snowflake, Terradata and Oracle databases - and make the files, objects and tables within them available to any tool or application running on PCAI.

In this section, you will learn how to make a data connection using the Delta Tables you created in Exercise 1. 

### Connect Delta Tables as Data Source using Apache Hive.

Let's take those Delta Tables you created in Exercise 1 and make them available to other applications in AI Essentials by connecting them as an **Data Source**. Apache Hive gives an SQL-like interface to query data stored in various databases and file systems, like the delta tables you created in Exercise 1. This will allow you to use EzPresto to turn them into datasets later in the exercise. 

<div class="alert alert-block alert-danger">
    <b>Important:</b> Set your <b>Username</b> here !
</div>

In [1]:
USERNAME="vince"

We start by defining a function to read the Kubernetes namespace from the service account mount point, allowing us to deploy resources in the correct namespace. If not running in a Kubernetes environment, it defaults to the "default" namespace.

In [2]:
def get_namespace_from_service_account():
    """
    Reads the Kubernetes namespace from the service account mount point.
    Returns 'default' if not running in a Kubernetes pod or if the file doesn't exist.
    """
    namespace_file = '/var/run/secrets/kubernetes.io/serviceaccount/namespace'
    try:
        with open(namespace_file, 'r') as f:
            return f.read().strip()
    except IOError:
        return 'default'

Set the Global variables required to run the exercise smootly

In [3]:
# Global configuration
NAMESPACE = get_namespace_from_service_account()
POSTGRES_PASSWORD = "postgres"
PG_SERVICE_NAME = f"{USERNAME}-retail-postgres"
PG_DATABASE_NAME = f"{USERNAME}-retail"

# Print the result
print("NAMESPACE:", NAMESPACE)
print("POSTGRES_PASSWORD:", POSTGRES_PASSWORD)
print("PG_SERVICE_NAME:", PG_SERVICE_NAME)
print("PG_DATABASE_NAME:", PG_DATABASE_NAME)

NAMESPACE: vincent-charbon-01f47b03
POSTGRES_PASSWORD: postgres
PG_SERVICE_NAME: vince-retail-postgres
PG_DATABASE_NAME: vince-retail


First let's get your details

In [4]:
def display_db_connection_details():
    """Display database connection details without connecting"""
    connection_details = {
        "Name": f"retail{USERNAME}",
        "Connection Url": f"jdbc:postgresql://{PG_SERVICE_NAME}.{NAMESPACE}.svc.cluster.local:5432/{PG_DATABASE_NAME}",
        "Connection User": "postgres",
        "Connection Password": POSTGRES_PASSWORD,
    }
    
    print("\nPostgreSQL Connection Details:")
    print("=" * 40)
    
    max_key_length = max(len(key) for key in connection_details.keys())
    
    for key, value in connection_details.items():
        print(f"{key.title():>{max_key_length}} : {value}")
    
    print("=" * 40 + "\n")

display_db_connection_details()


PostgreSQL Connection Details:
               Name : retailvince
     Connection Url : jdbc:postgresql://vince-retail-postgres.vincent-charbon-01f47b03.svc.cluster.local:5432/vince-retail
    Connection User : postgres
Connection Password : postgres



1. Navigate back to the AI Essentials dashboard.
1. In the sidebar navigation menu, select `Data Engineering` > `Data Sources`.
1. Under `Structured Data`, click `Add New Data Source`.
1. Under **PostgreSQL**, click `Create Connection`.

<img src="./images/exercise2/connect-dl.png" alt="Drawing" style="width: 25%;"/>

6. Under the `Structured Data` tab, you will now see your connected data source. 

### Viewing and Querying Data from Data Sources

Now that our Delta Tables are available via a Data Source, we can leverage the native data tools within AI Essentials to run queries and create datasets.

1. Click on the three dots in the top right hand corner of the newly created data source.
1. Click `Change to public access`. In the dialog box that appears, click `Proceed`.
1. Next, select `Query using Data Catalog`.

<img src="./images/exercise2/title.png" alt="Drawing" style="width: 25%;"/>

4. Under `Connected Data Sources`, look for the `retail` group.
5. Under the `retail` group, check the `default` box. 
6. Select the datasets for all three countries. 


<img src="./images/exercise2/datacatalog.png" alt="Drawing" style="width: 70%;"/>

7. Click `Selected Datasets` in the top left corner.
8. Click `Query Editor`.

Here, you are introduced to the the **Query Editor** where you can directly query data sources from specific datasets and data tables - all from within the HPE AI Essentials user interface! 

<img src="./images/exercise2/QueryEditor.png" alt="Drawing" style="width: 75%;"/>

9. Next, we're going to run an SQL query which will combine the data from our three tables (czech, germany, and swiss) in the retail.default schema. This will merge all columns from the czech and germany tables, and select specific columns from the swiss table whilst also applying a transformation to the country column. We'll also limit our final result set to 1000 rows. And all in a nice UI!

    Paste the following SQL Query into the `SQL Query` field, update the table names in the `FROM` to reflect with your environment and 

   Click `Run`.

```sql
SELECT 
  sc.customer_id,
  sc.customer_name,
  sc.customer_email,
  COUNT(DISTINCT so.order_id) AS order_frequency,
  SUM(sop.product_quantity * sca.price_cents) AS total_spend
FROM 
  retailvince.public.source_orders so
  JOIN retailvince.public.source_order_products sop ON so.order_id = sop.order_id
  JOIN retailvince.public.source_catalog sca ON sop.product_id = sca.product_id
  JOIN retailvince.public.source_customers sc ON so.customer_id = sc.customer_id
GROUP BY 
  sc.customer_id, sc.customer_name, sc.customer_email
ORDER BY 
  total_spend DESC, order_frequency DESC
LIMIT 10;
```

10. Expand the resulting query to visually validate it.
11. Under `Actions` (top-left corner of the table), click `Save as View`.

<img src="./images/exercise2/query-results.png" alt="Drawing" style="width: 75%;"/>


12. Name the View `retail`. 
13. We'll want to save the schema of this new table as a Custom Schema. Under `Schema`, select `+ Add new schema` and name it `retailschema`.

<img src="./images/exercise2/save-as-view.png" alt="Drawing" style="width: 45%;"/>

You have now saved the dataset resulting from your SQL query as an **Asset**. Assets are made available to other applications via EzPresto, as you will explore in Exercise 3. 

## **2. Cached Assets in HPE AI Essentials**

To view our saved Assets, including the one that we just created:

1. In AIE, under the sidebar navigation menu, select `Data Engineering` > `Cached Assets`. 
1. You should see an asset with Name `retail`. 
1. Click the three dots associated with the `retail` asset.

<img src="./images/exercise2/cachedasset.png" alt="Drawing" style="width: 75%;"/>

4. Click `View Columns`. 
5. Validate there are eleven fields. 

Should you ever wish to run futher queries on a Cached Asset in the future, check the box next to an asset and click `Query Editor`. 

## **3. Jupyter Magic Commands on HPE AI Essentials**

Jupyter Notebooks Magic functions, also known as magic commands or magics, are commands that you can execute within a code cell.   
Magics are not code of any language, but are shortcuts that extends the capabilities of a notebook. 

There are two types of magic commands - **Line** and **Cell** magic commands:

**Line magic** commands do not require a cell body and start with a single % character.  
**Cell magic** commands start with %% and require additional lines of input (a cell body). 

### HPE AI Essentials Magic Commands 
**HPE AI Essentials** supports both Line and Cell magic commands and includes custom commands that allow for users to interact with other tools native to AIE directly within notebooks.

We can check out the full list of custom HPE magic commands by running `%command`.

In [5]:
# The %commands command lists the magic commands and SDKs that are customized by Hewlett Packard Enterprise and are available in this notebook.
%commands

HTML(value='\n    <table style="border-collapse: collapse; border: 1px solid black; width: 100%;">\n        <t…

### Updating the Cached JWT Token

In a Jupyter notebook, a JWT token (JSON Web Token) is a compact and URL-safe means of representing authentication information to be transferred between other notebook servers or external applications. It is commonly used for securely authenticating and authorizing users within Jupyter environments, allowing them to access resources and execute code while ensuring their identity and permissions are properly validated.

When working in Jupyter notebooks for long durations, particularly when making calls to other applications, the JWT token can expire and result in an error when attempting to make calls. This is particularly relevant for working on a Jupyter notebook within **HPE AI Essentials**, which provides users to leverage a plethora of external tools within the notebook (Such as Spark, Livy and Presto).

**If you encounter a JWT token expiration error while running cells in a Jupyter notebook**, you can resolve it by running the `%update_token` magic command.  
This function updates the JWT in environment variables and any other locations where the token is utilized. 
  
Ideally, it is good practice to refresh the token prior to making external connections. Some examples relevant to the Smart Retail Experience demo include:

- Authentication when establishing a connection with PrestoDB.
- Authentication with local s3 minio object storage.  
- Authentication with KServe external API.  

In [6]:
%update_token

Token successfully refreshed.


### Directly interacting with connected SQL databases using the SQL Magic Command

Using the `%sql` magic command, you can directly query SQL databases from Data Sources you have connected to **HPE AI Essentials** from within Jupyter notebook cells! When you run the notebook cell containing `%sql` and your SQL query, the magic command sends the query to the database, runs the query, and retrieves the result.

This is made possible by the native integration of EzPresto into AIE. **However, the Data Source must be made publicly available.**

To change the access of a Data Source from `private` to `public`:

1. Navigate back to the AIE dashboard.
1. In the sidebar navigation menu, select `Data Engineering` > `Data Sources`.
1. Under the three dots in the top corner of the Data Source of interest, click `Change to public access`.

<div class="alert alert-block alert-danger">
<b>Important:</b> Wait until a confirmation message appears stating that the source is publicly available.
</div>

Now, let's try the `%sql` magic command to interact with our Delta Tables. 

In [7]:
%sql SELECT * FROM retailvince.public.source_customers limit 10

Running query in 'presto://ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local:8081'

SELECT * FROM retailvince.public.source_customers limit 10
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local'. Adding certifica

customer_id,customer_name,customer_surname,customer_email
1,None,Brown,liam.williams@example.com
2,Ava,Brown,noah.jones@example.com
3,None,Johnson,emma.johnson@example.com
4,Olivia,None,unknown.smith@example.com
5,Olivia,Smith,unknown.williams@example.com
6,Olivia,Van Helsing,unknown.smith@example.com
7,None,Brown,invalid.email
8,Noah,O'Brien,liam.jones@example.com
9,Olivia,Johnson,noah.brown@example.com
10,Ava,Williams,emma.smith@example.com


We can also save the output of our command as a Python variable!

In [8]:
result = %sql SELECT * FROM retailvince.public.source_customers limit 10
print(result)

Running query in 'presto://ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local:8081'

SELECT * FROM retailvince.public.source_customers limit 10
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


+-------------+---------------+------------------+------------------------------+
| customer_id | customer_name | customer_surname |        customer_email        |
+-------------+---------------+------------------+------------------------------+
|      1      |      None     |      Brown       |  liam.williams@example.com   |
|      2      |      Ava      |      Brown       |    noah.jones@example.com    |
|      3      |      None     |     Johnson      |   emma.johnson@example.com   |
|      4      |     Olivia    |       None       |  unknown.smith@example.com   |
|      5      |     Olivia    |      Smith       | unknown.williams@example.com |
|      6      |     Olivia    |   Van Helsing    |  unknown.smith@example.com   |
|      7      |      None     |      Brown       |        invalid.email         |
|      8      |      Noah     |     O'Brien      |    liam.jones@example.com    |
|      9      |     Olivia    |     Johnson      |    noah.brown@example.com    |
|      10     | 

/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


# **Conclusion**

In this exercise, you learned how **EzPresto** on **HPE AI Essentials** makes connecting internal and external data sources to your applications, such this notebook hosted on AIE, a snap. You learned how to leverage the data engineering tools available within the AIE interface, including the Query Editor, to create datasets from your data sources that could then be shared and quered using HPE Magic Commands inside AIE-hosted Jupyter Notebooks.

If you look at the results of the SQL query, you can see some inconsistencies. Therefore, in the next exercise, you will learn how to transform this data using **Apache Spark** to prepare it for AI!

Next: <a href="./03.explore_data_with_spark.ipynb" style="color: black"><b style="color: #01a982;">Exercise 3:</b> Explore Retail Data with Apache Spark</a>

Previous: <a href="./01.prepare_the_data.ipynb" style="color: black"><b style="color: #01a982;">Exercise 1:</b> Prepare the Data</a>